In [1]:
from datetime import datetime
import MetaTrader5 as mt5
#import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
#connect to mt5
if not mt5.initialize():
   print("Unable to connect to MT5")
   mt5.shutdown()

In [3]:
symbol_info = mt5.symbol_info("EURUSD")
print("EurUsd: ")
print("Spread: ", symbol_info.spread, " Digits: ", symbol_info.digits)
print("Bid: ", symbol_info.bid, " Ask: ", symbol_info.ask)

EurUsd: 
Spread:  7  Digits:  5
Bid:  1.19044  Ask:  1.19051


In [4]:
date = datetime.now().strftime("%Y %m %d %H %M").split()
date[3] = (int(date[3]) + 3) % 24
for num, d in enumerate(date):
   date[num] = int(d)

eurusd_rates = mt5.copy_rates_from("EURUSD", mt5.TIMEFRAME_M5, datetime(date[0],date[1],date[2],date[3],date[4]), 10080)
# for eurusd in eurusd_rates:
#    print(datetime.fromtimestamp(eurusd[0]))
print(type(eurusd_rates))
print(eurusd_rates)

<class 'numpy.ndarray'>
[(1611958800, 1.21346, 1.21349, 1.21331, 1.21347,  96, 0, 0)
 (1611959100, 1.21347, 1.2137 , 1.21337, 1.21347, 133, 0, 0)
 (1611959400, 1.21347, 1.21374, 1.21346, 1.21348, 215, 0, 0) ...
 (1616193600, 1.19034, 1.19038, 1.19029, 1.19033,  45, 1, 0)
 (1616193900, 1.19033, 1.19035, 1.19021, 1.19023,  97, 1, 0)
 (1616194200, 1.19023, 1.19055, 1.19021, 1.19044, 179, 1, 0)]


In [5]:
test = np.array(eurusd_rates, dtype=[('timestamp', 'int64'), ("open", "float64"), ("high", "float64"),
    ("low", "float64"), ("close", "float64"), ("volume", "int64"), ("spread", "int64"), ("real_volume", "int64")])
print("Spread:")
print("moy: ", test["spread"].mean(), " std: ", test["spread"].std(), " min: ", test["spread"].min(), " max: ", test["spread"].max(), " per25: ", np.percentile(test["spread"], 25), " median: ", np.percentile(test["spread"], 50), " per75: ", np.percentile(test["spread"], 75))

Spread:
moy:  1.3281746031746031  std:  3.1483240441501654  min:  0  max:  56  per25:  1.0  median:  1.0  per75:  1.0


In [6]:
import talib
import pandas as pd

rsi = talib.RSI(test["close"], timeperiod=14)
print(len(rsi))
print(rsi[:20])

10080
[        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan 47.25274725 44.8635634  40.67957904 47.22690428
 52.8332552  41.17005173]


In [7]:
currency = "USDJPY" # "GBPUSD" "EURCHF" "EURUSD"
eurusd_M5 = mt5.copy_rates_from_pos(currency, mt5.TIMEFRAME_M5, 0, 1_000) # 99_999
eurusd_M30 = mt5.copy_rates_from_pos(currency, mt5.TIMEFRAME_M30, 0, 1_000) # 99_999

print(eurusd_M5[-1])
print(eurusd_M30[-1])
#mt5.last_error()
pd_eurusd_M5 = pd.DataFrame(eurusd_M5)
pd_eurusd_M30 = pd.DataFrame(eurusd_M30)


cci5 = talib.CCI(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"], timeperiod= 14)

print("CCI  M5: ", cci5.iloc[-1])

(1616194200, 108.904, 108.907, 108.878, 108.879, 356, 0, 0)
(1616193000, 108.904, 108.921, 108.878, 108.879, 687, 0, 0)
CCI  M5:  -82.67589388701698


In [8]:
# Bollinger Band
pd_eurusd_M5["upperBBands_M5"], pd_eurusd_M5["middleBBands_M5"], pd_eurusd_M5["lowerBBands_M5"] = talib.BBANDS(pd_eurusd_M5["close"], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)

# Double Exponential Moving Average
pd_eurusd_M5["DEMA_M5"] = talib.DEMA(pd_eurusd_M5["close"], timeperiod= 21)

# MidPoint over period
pd_eurusd_M5["Midpoint_M5"] = talib.MIDPOINT(pd_eurusd_M5["close"], timeperiod= 14)

# Midpoint Price over period
pd_eurusd_M5["Midprice_M5"] = talib.MIDPRICE(pd_eurusd_M5["high"], pd_eurusd_M5["low"], timeperiod= 14)

# Parabolic SAR
pd_eurusd_M5["SAR_M5"] = talib.SAR(pd_eurusd_M5["high"], pd_eurusd_M5["low"], acceleration=0.02, maximum=0.2)

# Aroon Oscillator
pd_eurusd_M5["Aroon_M5"] = talib.AROONOSC(pd_eurusd_M5["high"], pd_eurusd_M5["low"], timeperiod=14)

# Balance Of Power
pd_eurusd_M5["BOP_M5"] = talib.BOP(pd_eurusd_M5["open"], pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"])

# CCI
pd_eurusd_M5["CCI_M5"] = talib.CCI(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"], timeperiod= 14)

# MACD
pd_eurusd_M5["MACD_M5"], pd_eurusd_M5["MACDSignal_M5"], pd_eurusd_M5["MACDHist_M5"] = talib.MACD(pd_eurusd_M5["close"], fastperiod=12, slowperiod=26, signalperiod=9)

# Momentum
pd_eurusd_M5["Mom_M5"] = talib.MOM(pd_eurusd_M5["close"], timeperiod=10)

# Percentage Price Oscillator
pd_eurusd_M5["PPO_M5"] = talib.PPO(pd_eurusd_M5["close"], fastperiod=12, slowperiod=26, matype=0)

# Rate of change
pd_eurusd_M5["ROC_M5"] = talib.ROC(pd_eurusd_M5["close"], timeperiod=10)

# Relative Strength Index
pd_eurusd_M5["RSI_M5"] = talib.RSI(pd_eurusd_M5["close"], timeperiod=14)

# Stochastic
pd_eurusd_M5["Stoc_slowk_M5"], pd_eurusd_M5["Stoc_slowd_M5"] = talib.STOCH(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

# TRIX
pd_eurusd_M5["TRIX_M5"] = talib.TRIX(pd_eurusd_M5["close"], timeperiod=30)

# Ultimate Oscillator
pd_eurusd_M5["UltOsc_M5"] = talib.ULTOSC(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"], timeperiod1=7, timeperiod2=14, timeperiod3=28)

# Williams' %R
pd_eurusd_M5["WillamsR_M5"] = talib.WILLR(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"], timeperiod=14)

# Chaikin A/D Oscillator
pd_eurusd_M5["ADOsc_M5"] = talib.ADOSC(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"], pd_eurusd_M5["tick_volume"], fastperiod=3, slowperiod=10)

# On Balance Volume
pd_eurusd_M5["OBV_M5"] = talib.OBV(pd_eurusd_M5["close"], pd_eurusd_M5["tick_volume"])

# True Range
pd_eurusd_M5["TrueRange_M5"] = talib.TRANGE(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"])

In [9]:
#Merge M5 and M30 Dataframes
pd_eurusd = pd_eurusd_M5.merge(pd_eurusd_M30, how="left", left_on= "time", right_on= "time")

# print(pd_eurusd[:40])
# test = pd.DataFrame(pd_eurusd, columns=["time", "high_x", "low_x", "SAR_M5"])
# print(test[:40])
# prev_SAR = 1.098328
# prev_af = 0.04
# EP = 1.09842
# print(prev_SAR + prev_af * (EP - prev_SAR))

In [10]:
# Remove first empty lines of M30

for i in range(0,7):
    if pd_eurusd["time"].iloc[0] % 1800 == 0:
        break
    else:
        pd_eurusd = pd_eurusd.drop([i])

# pd_eurusd[0:12]

In [11]:
# Adjust M30 open, high, low, close values

for i in range(0, len(pd_eurusd)):
    if pd_eurusd["time"].iloc[i] % 1800 == 0:
        pd_eurusd["open_y"].iloc[i] = pd_eurusd["open_x"].iloc[i]
        pd_eurusd["high_y"].iloc[i] = pd_eurusd["high_x"].iloc[i]
        pd_eurusd["low_y"].iloc[i] = pd_eurusd["low_x"].iloc[i]
        pd_eurusd["close_y"].iloc[i] = pd_eurusd["close_x"].iloc[i]
    else:
        pd_eurusd["open_y"].iloc[i] = pd_eurusd["open_y"].iloc[i-1]
        pd_eurusd["close_y"].iloc[i] = pd_eurusd["close_x"].iloc[i]
        if pd_eurusd["high_x"].iloc[i] > pd_eurusd["high_y"].iloc[i-1]:
            pd_eurusd["high_y"].iloc[i] = pd_eurusd["high_x"].iloc[i]
        else:
            pd_eurusd["high_y"].iloc[i] = pd_eurusd["high_y"].iloc[i-1]
        if pd_eurusd["low_x"].iloc[i] < pd_eurusd["low_y"].iloc[i-1]:
            pd_eurusd["low_y"].iloc[i] = pd_eurusd["low_x"].iloc[i]
        else:
            pd_eurusd["low_y"].iloc[i] = pd_eurusd["low_y"].iloc[i-1]


# pd_eurusd[0:10]

c:\Users\nanar\Documents\Python\Metatrader\MT5\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [12]:
# Adjust M30 volume

for i in range(0, len(pd_eurusd)):
    if pd_eurusd["time"].iloc[i] % 1800 == 0:
        pd_eurusd["tick_volume_y"].iloc[i] = pd_eurusd["tick_volume_x"].iloc[i]
    else:
        pd_eurusd["tick_volume_y"].iloc[i] = pd_eurusd["tick_volume_x"].iloc[i] + pd_eurusd["tick_volume_y"].iloc[i-1]

# pd_eurusd[0:15]

In [13]:
# Bollinger bands M30
timeperiod = 20
nbdev = 2

count = 0
midBand = np.array([])
upBand = np.array([])
lowBand = np.array([])
tpTemp = np.array([])
#closeTemp = np.array([])


for i in range(0, len(pd_eurusd)):
    if count < timeperiod-1:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            count += 1
            tp = (pd_eurusd["high_y"].iloc[i] + pd_eurusd["low_y"].iloc[i] + pd_eurusd["close_y"].iloc[i]) / 3
            tpTemp = np.append(tpTemp, tp)
            #closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            midBand = np.append(midBand, [np.nan])
            upBand = np.append(upBand, [np.nan])
            lowBand = np.append(lowBand, [np.nan])
        else:
            midBand = np.append(midBand, [np.nan])
            upBand = np.append(upBand, [np.nan])
            lowBand = np.append(lowBand, [np.nan])
    else:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            # Calcul midBand, return midBand and new smaTemp
            # Calcul upBand, return upBand
            # Calcul lowBand, return lowBand and new closeTemp
            # Calcul midBand, return midBand
            tp = (pd_eurusd["high_y"].iloc[i] + pd_eurusd["low_y"].iloc[i] + pd_eurusd["close_y"].iloc[i]) / 3
            tpTemp = np.append(tpTemp, tp)
            smaTemp = talib.SMA(tpTemp, timeperiod= timeperiod)
            midBand = np.append(midBand, [smaTemp[-1]])
            # Calcul sigma
            std_tp = np.std(tpTemp)
            # Calcul upBand, return upBand
            temp_up = smaTemp[-1] + nbdev * std_tp
            upBand = np.append(upBand, [temp_up])
            # Calcul lowBand, return lowBand
            temp_low = smaTemp[-1] - nbdev * std_tp
            lowBand = np.append(lowBand, [temp_low])
            # Delete first tpTemp value to stay with tiemperiod-1 values
            tpTemp = np.delete(tpTemp, 0)
        else:
            # Calcul midBand, return midBand
            tp = (pd_eurusd["high_y"].iloc[i] + pd_eurusd["low_y"].iloc[i] + pd_eurusd["close_y"].iloc[i]) / 3
            temp_tp = np.append(tpTemp, tp)
            smaTemp = talib.SMA(temp_tp, timeperiod= timeperiod)
            midBand = np.append(midBand, [smaTemp[-1]])
            # Calcul sigma
            std_tp = np.std(temp_tp)
            # Calcul upBand, return upBand
            temp_up = smaTemp[-1] + nbdev * std_tp
            upBand = np.append(upBand, [temp_up])
            # Calcul lowBand, return lowBand
            temp_low = smaTemp[-1] - nbdev * std_tp
            lowBand = np.append(lowBand, [temp_low])

pd_eurusd["upperBBands_M30"] = upBand
pd_eurusd["middleBBands_M30"] = midBand
pd_eurusd["lowerBBands_M30"] = lowBand

In [14]:
# Double Exponential Moving Average ** On Close ** timeperiod = 21
timeperiod = 21

count = 0
dema = np.array([])
closeTemp = np.array([])
emaTemp = np.array([])

for i in range(0, len(pd_eurusd)):
    if count < timeperiod-1:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            count += 1
            closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            dema = np.append(dema, np.nan)
        else:
            dema = np.append(dema, np.nan)
    else:
        if count < 2 * timeperiod -1:
            if pd_eurusd["time"].iloc[i] % 1800 == 1500:
                count += 1
                closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
                ema1 = talib.EMA(closeTemp, timeperiod= timeperiod)
                emaTemp = np.append(emaTemp, ema1[-1])
                closeTemp = np.delete(closeTemp, 0)
                dema = np.append(dema, np.nan)
            else:
                dema = np.append(dema, np.nan)
        else:
            if pd_eurusd["time"].iloc[i] % 1800 == 1500:
                closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
                ema1 = talib.EMA(closeTemp, timeperiod= timeperiod)
                emaTemp = np.append(emaTemp, ema1[-1])
                ema2 = talib.EMA(emaTemp, timeperiod= timeperiod)
                calc = 2 * ema1[-1] - ema2[-1]
                dema = np.append(dema, calc)
                closeTemp = np.delete(closeTemp, 0)
                emaTemp = np.delete(emaTemp, 0)
            else:
                c_temp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
                ema1 = talib.EMA(c_temp, timeperiod= timeperiod)
                ema_tp = np.append(emaTemp, ema1[-1])
                ema2 = talib.EMA(ema_tp, timeperiod= timeperiod)
                calc = 2 * ema1[-1] - ema2[-1]
                dema = np.append(dema, calc)

pd_eurusd["DEMA_M30"] = dema

In [15]:
# MidPoint over period * on close * timeperiod = 14
timeperiod = 14

count = 0
midpoint = np.array([])
closeTemp = np.array([])

for i in range(0, len(pd_eurusd)):
    if count < timeperiod-1:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            count += 1
            closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            midpoint = np.append(midpoint, np.nan)
        else:
            midpoint = np.append(midpoint, np.nan)
    else:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            calc = (closeTemp.max() + closeTemp.min()) / 2
            midpoint = np.append(midpoint, calc)
            closeTemp = np.delete(closeTemp, 0)
        else:
            c_temp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            calc = (c_temp.max() + c_temp.min()) / 2
            midpoint = np.append(midpoint, calc)

pd_eurusd["Midpoint_M30"] = midpoint

In [16]:
# Midpoint Price over period * on high, low * timeperiod = 14
timeperiod = 14

count = 0
midpoint = np.array([])
highTemp = np.array([])
lowTemp = np.array([])

for i in range(0, len(pd_eurusd)):
    if count < timeperiod-1:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            count += 1
            highTemp = np.append(highTemp, pd_eurusd["high_y"].iloc[i])
            lowTemp = np.append(lowTemp, pd_eurusd["low_y"].iloc[i])
            midpoint = np.append(midpoint, np.nan)
        else:
            midpoint = np.append(midpoint, np.nan)
    else:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            highTemp = np.append(highTemp, pd_eurusd["high_y"].iloc[i])
            lowTemp = np.append(lowTemp, pd_eurusd["low_y"].iloc[i])
            calc = (highTemp.max() + lowTemp.min()) / 2
            midpoint = np.append(midpoint, calc)
            highTemp = np.delete(highTemp, 0)
            lowTemp = np.delete(lowTemp, 0)
        else:
            h_temp = np.append(highTemp, pd_eurusd["high_y"].iloc[i])
            l_temp = np.append(lowTemp, pd_eurusd["low_y"].iloc[i])
            calc = (h_temp.max() + l_temp.min()) / 2
            midpoint = np.append(midpoint, calc)

pd_eurusd["Midprice_M30"] = midpoint

In [17]:
# Parabolic SAR * acceleration = 0.02, maximum = 0.2


acc = 0.02
maximum = 0.2

trend = 0 # uptrend => 1, donwtrend => -1
SAR = np.array([])
ext_point = 0
prev_pos = 0

i_start = 0
init_bool = True

# initialisation, get trend

for i in range(0, len(pd_eurusd)):
    if pd_eurusd["time"].iloc[i] % 1800 == 1500:
        if init_bool:
            i_start = i
            SAR = np.append(SAR, np.nan)
            init_bool = False
        else:
            # uptrend
            if pd_eurusd["high_y"].iloc[i] > pd_eurusd["high_y"].iloc[i_start] and pd_eurusd["low_y"].iloc[i] > pd_eurusd["low_y"].iloc[i_start]:
                trend = 1
                SAR = np.append(SAR, pd_eurusd["low_y"].iloc[i_start])
                ext_point = pd_eurusd["low_y"].iloc[i_start]
                i_start = i
                prev_pos = i

                break 
            # donwtrend
            elif pd_eurusd["high_y"].iloc[i] < pd_eurusd["high_y"].iloc[i_start] and pd_eurusd["low_y"].iloc[i] < pd_eurusd["low_y"].iloc[i_start]:
                trend = -1
                SAR = np.append(SAR, pd_eurusd["high_y"].iloc[i_start])
                ext_point = pd_eurusd["high_y"].iloc[i_start]
                i_start = i
                prev_pos = i
                break
            # no trend
            else:
                trend = 0
                SAR = np.append(SAR, np.nan)
                i_start = i
    # else if not good time => nan
    else:
        SAR = np.append(SAR, np.nan)


for i in range(i_start+1, len(pd_eurusd)):
    # check change of trend
    #   => if high or low above or below SAR -> change trend and new SAR == highest high or lowest low

    # else calculate SAR
    if pd_eurusd["time"].iloc[i] % 1800 == 1500:
        # change of trend
        if trend == 1 and pd_eurusd["low_y"].iloc[i] <= SAR[prev_pos]:
            trend = -1
            SAR = np.append(SAR, ext_point)
            ext_point = pd_eurusd["low_y"].iloc[i]
            acc = 0.02
            prev_pos = i
            continue
        elif trend == -1 and pd_eurusd["high_y"].iloc[i] >= SAR[prev_pos]:
            trend = 1
            SAR = np.append(SAR, ext_point)
            ext_point = pd_eurusd["high_y"].iloc[i]
            acc = 0.02
            prev_pos = i
            continue

        # no change of trend => calc SAR
        if trend == 1:
            new_SAR = SAR[prev_pos] + acc * (ext_point - SAR[prev_pos])
            SAR = np.append(SAR, new_SAR)
            if pd_eurusd["high_y"].iloc[i] > ext_point:
                ext_point = pd_eurusd["high_y"].iloc[i]
                if acc < maximum:
                    acc += 0.02
            prev_pos = i
        elif trend == -1:
            new_SAR = SAR[prev_pos] - acc * (SAR[prev_pos] - ext_point)
            SAR = np.append(SAR, new_SAR)
            if pd_eurusd["low_y"].iloc[i] < ext_point:
                ext_point = pd_eurusd["low_y"].iloc[i]
                if acc < maximum:
                    acc += 0.02
            prev_pos = i
    
    else:
        # # change of trend
        
        # no change of trend => calc SAR
        if trend == 1:
            new_SAR = SAR[prev_pos] + acc * (ext_point - SAR[prev_pos])
            SAR = np.append(SAR, new_SAR)
            # if pd_eurusd["high_y"].iloc[i] > ext_point:
            #     ext_point = pd_eurusd["high_y"].iloc[i]
            #     # if acc < maximum:
            #     #     acc += 0.02
        elif trend == -1:
            new_SAR = SAR[prev_pos] - acc * (SAR[prev_pos] - ext_point)
            SAR = np.append(SAR, new_SAR)
            # if pd_eurusd["low_y"].iloc[i] < ext_point:
            #     ext_point = pd_eurusd["low_y"].iloc[i]
            #     # if acc < maximum:
            #     #     acc += 0.02



pd_eurusd["SAR_M30"] = SAR

In [18]:
# Aroon Oscillator * timeperiod = 14

timeperiod = 14
count = 0

highTemp = np.array([])
lowTemp = np.array([])
arrOsc = np.array([])

for i in range(0, len(pd_eurusd)):
    if count < timeperiod-1:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            count += 1
            highTemp = np.append(highTemp, pd_eurusd["high_y"].iloc[i])
            lowTemp = np.append(lowTemp, pd_eurusd["low_y"].iloc[i])
            arrOsc = np.append(arrOsc, np.nan)
        else:
            arrOsc = np.append(arrOsc, np.nan)
    else:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            highTemp = np.append(highTemp, pd_eurusd["high_y"].iloc[i])
            lowTemp = np.append(lowTemp, pd_eurusd["low_y"].iloc[i])
            arrUp = (np.argmax(highTemp) / timeperiod) * 100
            arrDown = (np.argmin(lowTemp) / timeperiod) * 100
            calc = arrUp - arrDown
            arrOsc = np.append(arrOsc, calc)
            highTemp = np.delete(highTemp, 0)
            lowTemp = np.delete(lowTemp, 0)
        else:
            h_temp = np.append(highTemp, pd_eurusd["high_y"].iloc[i])
            l_temp = np.append(lowTemp, pd_eurusd["low_y"].iloc[i])
            arrUp = (np.argmax(h_temp) / timeperiod) * 100
            arrDown = (np.argmin(l_temp) / timeperiod) * 100
            calc = arrUp - arrDown
            arrOsc = np.append(arrOsc, calc)

pd_eurusd["Aroon_M30"] = arrOsc

In [19]:
# Balance Of Power

bop = np.array([])

for i in range(0, len(pd_eurusd)):
    calc = (pd_eurusd["close_y"].iloc[i] - pd_eurusd["open_y"]).iloc[i] / (pd_eurusd["high_y"].iloc[i] - pd_eurusd["low_y"].iloc[i])
    bop = np.append(bop, calc)

pd_eurusd["BOP_M30"] = bop

In [20]:
# CCI * timeperiod = 14

timeperiod = 14
count = 0

typ_price = np.array([])
cci = np.array([])

for i in range(0, len(pd_eurusd)):
    if count < timeperiod-1:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            count += 1
            calc_TP = (pd_eurusd["high_y"].iloc[i] + pd_eurusd["low_y"].iloc[i] + pd_eurusd["close_y"].iloc[i]) / 3
            typ_price = np.append(typ_price, calc_TP)
            cci = np.append(cci, np.nan)
        else:
            cci = np.append(cci, np.nan)
    else:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            calc_TP = (pd_eurusd["high_y"].iloc[i] + pd_eurusd["low_y"].iloc[i] + pd_eurusd["close_y"].iloc[i]) / 3
            typ_price = np.append(typ_price, calc_TP)
            sma_TP = talib.SMA(typ_price, timeperiod = timeperiod)
            mean_dev = np.mean(np.absolute(typ_price - sma_TP[-1]))
            calc = (typ_price[-1] - sma_TP[-1]) / (0.015 * mean_dev)
            cci = np.append(cci, calc)
            typ_price = np.delete(typ_price, 0)
        else:
            calc_TP = (pd_eurusd["high_y"].iloc[i] + pd_eurusd["low_y"].iloc[i] + pd_eurusd["close_y"].iloc[i]) / 3
            typ_p = np.append(typ_price, calc_TP)
            sma_TP = talib.SMA(typ_p, timeperiod = timeperiod)
            mean_dev = np.mean(np.absolute(typ_p - sma_TP[-1]))
            calc = (typ_p[-1] - sma_TP[-1]) / (0.015 * mean_dev)
            # print(mean_dev)
            cci = np.append(cci, calc)

pd_eurusd["CCI_M30"] = cci

In [21]:
# MACD *** fastperiod=12, slowperiod=26, signalperiod=9
fastperiod = 12
slowperiod = 26
signalperiod = 9
count = 0

closeTemp = np.array([])
macd_tp = np.array([])
macd = np.array([])
signal = np.array([])
hist = np.array([])

for i in range(0, len(pd_eurusd)):
    if count < slowperiod -1:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            count +=1
            closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            macd = np.append(macd, np.nan)
            signal = np.append(signal, np.nan)
            hist = np.append(hist, np.nan)
        else:
            macd = np.append(macd, np.nan)
            signal = np.append(signal, np.nan)
            hist = np.append(hist, np.nan)

    elif count < (slowperiod + signalperiod -2):
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            count += 1
            closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            ema_f = talib.EMA(closeTemp, timeperiod = fastperiod)
            ema_s = talib.EMA(closeTemp, timeperiod = slowperiod)
            calc_macd = ema_f[-1] - ema_s[-1]
            macd_tp = np.append(macd_tp, calc_macd)
            macd = np.append(macd, calc_macd)
            signal = np.append(signal, np.nan)
            hist = np.append(hist, np.nan)
            # closeTemp = np.delete(closeTemp, 0)
        else:
            close_tp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            ema_f = talib.EMA(close_tp, timeperiod = fastperiod)
            ema_s = talib.EMA(close_tp, timeperiod = slowperiod)
            calc_macd = ema_f[-1] - ema_s[-1]
            macd = np.append(macd, calc_macd)
            signal = np.append(signal, np.nan)
            hist = np.append(hist, np.nan)
    else:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            ema_f = talib.EMA(closeTemp, timeperiod = fastperiod)
            ema_s = talib.EMA(closeTemp, timeperiod = slowperiod)
            calc_macd = ema_f[-1] - ema_s[-1]
            macd_tp = np.append(macd_tp, calc_macd)
            macd = np.append(macd, calc_macd)
            calc_sign = talib.EMA(macd_tp, timeperiod = signalperiod)
            signal = np.append(signal, calc_sign[-1])
            h_temp = calc_macd - calc_sign[-1]
            hist = np.append(hist, h_temp)
            # closeTemp = np.delete(closeTemp, 0)
            macd_tp = np.delete(macd_tp, 0)
        else:
            close_tp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            ema_f = talib.EMA(close_tp, timeperiod = fastperiod)
            ema_s = talib.EMA(close_tp, timeperiod = slowperiod)
            calc_macd = ema_f[-1] - ema_s[-1]
            macd = np.append(macd, calc_macd)
            macdTemp = np.append(macd_tp, calc_macd)
            calc_sign = talib.EMA(macdTemp, timeperiod = signalperiod)
            signal = np.append(signal, calc_sign[-1])
            h_temp = calc_macd - calc_sign[-1]
            hist = np.append(hist, h_temp)

pd_eurusd["MACD_M30"] = macd
pd_eurusd["MACDSignal_M30"] = signal
pd_eurusd["MACDHist_M30"] = hist

In [22]:
# Momentum * timeperiod = 10
timeperiod = 10
count = 0

momentum = np.array([])
closeTemp = np.array([])

for i in range(0, len(pd_eurusd)):
    if count < timeperiod:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            count += 1
            closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            momentum = np.append(momentum, np.nan)
        else:
            momentum = np.append(momentum, np.nan)
    else:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            calc = 100 * (closeTemp[-1] / closeTemp[0])
            momentum = np.append(momentum, calc)
            closeTemp = np.delete(closeTemp, 0)
        else:
            close_tp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            calc = 100 * (close_tp[-1] / close_tp[0])
            momentum = np.append(momentum, calc)

pd_eurusd["Mom_M30"] = momentum

In [23]:
pd_eurusd.describe()
# test = pd.DataFrame(pd_eurusd, columns=["time", "close_x", "close_y"])
# print(test[test["time"] % 1800 == 1500][-50:])
# print(test[-50:])


,time,open_x,high_x,low_x,close_x,tick_volume_x,spread_x,real_volume_x,upperBBands_M5,middleBBands_M5,...,Midpoint_M30,Midprice_M30,SAR_M30,Aroon_M30,BOP_M30,CCI_M30,MACD_M30,MACDSignal_M30,MACDHist_M30,Mom_M30
count,9.950000e+02,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.0,981.000000,981.000000,...,917.000000,917.000000,984.000000,917.000000,995.000000,917.000000,845.000000,797.000000,797.000000,935.000000
mean,1.616045e+09,108.992470,109.008686,108.975221,108.992080,240.845226,0.656281,0.0,109.069591,108.992079,...,108.995829,108.993523,108.988773,6.768967,0.028156,12.296259,-0.005749,-0.006384,-0.000574,99.993006
std,8.621282e+04,0.132488,0.131663,0.133653,0.132411,161.083667,3.728250,0.0,0.132546,0.123510,...,0.102502,0.100641,0.174990,51.980155,0.534717,98.263179,0.034888,0.032067,0.021558,0.144978
min,1.615896e+09,108.613000,108.644000,108.609000,108.613000,10.000000,0.000000,0.0,108.743718,108.677200,...,108.754000,108.767500,108.609000,-85.714286,-1.000000,-298.252549,-0.100662,-0.080614,-0.069185,99.493849
25%,1.615971e+09,108.883000,108.896000,108.869000,108.883000,138.000000,0.000000,0.0,108.953978,108.888800,...,108.927000,108.914500,108.837953,-35.714286,-0.430769,-61.906594,-0.031348,-0.029349,-0.014395,99.927991
50%,1.616045e+09,108.992000,109.007000,108.976000,108.992000,204.000000,0.000000,0.0,109.064061,108.991150,...,108.984000,108.987000,109.023194,7.142857,0.047619,21.444072,0.000407,-0.003600,0.001819,100.010092
75%,1.616120e+09,109.093500,109.108000,109.078000,109.093500,293.000000,0.000000,0.0,109.171055,109.092350,...,109.079000,109.072500,109.110984,57.142857,0.488034,81.927610,0.025973,0.024804,0.010073,100.080710
max,1.616194e+09,109.309000,109.326000,109.293000,109.308000,1562.000000,32.000000,0.0,109.437613,109.266600,...,109.227000,109.212000,109.326000,92.857143,1.000000,267.300800,0.047771,0.036122,0.045793,100.410126


In [24]:
# Test fractal *** signalperiod = 15, signalsmaperiod = 30
signalperiod = 15
signalsmaperiod = 30

fractal = np.array([])
signal = np.array([])
signalsma = np.array([])
typ_priceM30 = np.array([])
typ_priceM5 = np.array([])
diffM30 = np.array([])
diffM5 = np.array([])

for i in range(0, len(pd_eurusd)):
    typM5 = (pd_eurusd["high_x"].iloc[i] + pd_eurusd["low_x"].iloc[i] + pd_eurusd["close_x"].iloc[i]) / 3
    typM30 = (pd_eurusd["high_y"].iloc[i] + pd_eurusd["low_y"].iloc[i] + pd_eurusd["close_y"].iloc[i]) / 3
    typ_priceM5 = np.append(typ_priceM5, typM5)
    typ_priceM30 = np.append(typ_priceM30, typM30)

start_bool = False
for i in range(0, len(pd_eurusd)):
    if i == 0:
        diffM5 = np.append(diffM5, np.nan)
        diffM30 = np.append(diffM30, np.nan)
        continue
    if pd_eurusd["time"].iloc[i] % 1800 == 1500:
        if not start_bool:
            start_bool = True
            diffM5 = np.append(diffM5, np.abs(typ_priceM5[i] - typ_priceM5[i-1]))
            diffM30 = np.append(diffM30, np.nan)
            prev_pos = i
        else:
            diffM5 = np.append(diffM5, np.abs(typ_priceM5[i] - typ_priceM5[i-1]))
            diffM30 = np.append(diffM30, np.abs(typ_priceM30[i] - typ_priceM30[prev_pos]))
            prev_pos = i
    else:
        if not start_bool:
            diffM5 = np.append(diffM5, np.abs(typ_priceM5[i] - typ_priceM5[i-1]))
            diffM30 = np.append(diffM30, np.nan)
        else:
            diffM5 = np.append(diffM5, np.abs(typ_priceM5[i] - typ_priceM5[i-1]))
            diffM30 = np.append(diffM30, np.abs(typ_priceM30[i] - typ_priceM30[prev_pos]))

q = 0
p = 1
sum = 0
start_bool = False
for i in range(0, len(pd_eurusd)):
    if i < len([i for i in np.isnan(diffM30) if i]):
        fractal = np.append(fractal, np.nan)
        continue
    if pd_eurusd["time"].iloc[i] % 1800 == 1500:
        if not start_bool:
            start_bool = True
            prev_pos = i
            fractal = np.append(fractal, np.nan)
        else:
            sum += diffM5[i]
            if sum == 0 or diffM30[i] == 0:
                fractal = np.append(fractal, 0.0)
            else:
                q = sum / diffM30[i]
                if (q == 1):
                    fractal = np.append(fractal, 0.0)
                else:
                    calc = math.log(p) / math.log(q)
                    fractal = np.append(fractal, calc)
            p = 1
            sum = 0
    else:
        if not start_bool:
            fractal = np.append(fractal, np.nan)
        else:
            sum += diffM5[i]
            if sum == 0 or diffM30[i] == 0:
                fractal = np.append(fractal, 0.0)
            else:
                q = sum / diffM30[i]
                if (q == 1):
                    fractal = np.append(fractal, 0.0)
                else:
                    calc = math.log(p) / math.log(q)
                    fractal = np.append(fractal, calc)
            p += 1

signal = talib.SMA(fractal, timeperiod = signalperiod)
signalsma = talib.SMA(fractal, timeperiod = signalsmaperiod)

pd_eurusd["CustFractSignal"] = signal
pd_eurusd["CustFractSignalSMA"] = signalsma


In [25]:
print(len([i for i in np.isnan(signalsma) if i]))

41


In [26]:
timer = [0]
curr = 0
prev = 0

for i in range(1, len(pd_eurusd)):
    if pd_eurusd["time"].iloc[i] != pd_eurusd["time"].iloc[i-1] + 300:
        timer.append(i)
        curr = pd_eurusd["time"].iloc[i]
        prev = pd_eurusd["time"].iloc[i-1]

timer.append(len(pd_eurusd))

print(timer)
print(curr)
print(prev)
print("***")
from datetime import datetime
# for i in range(7255, 7257):
#     print(datetime.fromtimestamp(pd_eurusd["time"].iloc[i]), " *** ", pd_eurusd["open_x"].iloc[i], " *** ", pd_eurusd["high_x"].iloc[i], " *** ", pd_eurusd["low_x"].iloc[i], " *** ", pd_eurusd["close_x"].iloc[i])

print(datetime.fromtimestamp(pd_eurusd["time"].iloc[0]))
print(datetime.fromtimestamp(pd_eurusd["time"].iloc[len(pd_eurusd)-1]))

[0, 995]
0
0
***
2021-03-16 13:00:00
2021-03-19 23:50:00


In [27]:
# test = pd.DataFrame(pd_eurusd, columns=["time", "close_x", "CustFractSignal", "CustFractSignalSMA"])
# # print(test[test["time"] % 1800 == 1500][-50:])
# print(test[-50:])

def pivot(high, low, close):
    point = (high + low + close) / 3
    R1 = point + 0.382 * (high - low) # (2 * point) - low
    S1 = point - 0.382 * (high - low) # (2 * point) - high
    return R1, S1

capital = 100
lot = 0.01

levier = 30
mise = round(100_000 * lot / 30, 2)
spread = 2
# current_pos_i = 0
# current_pos_j = 0
gains = np.array([])
truc = []
test = []
trades = 0

end = []
for _ in range(1, len(timer)):
    for i in range(timer[_-1]+52, timer[_]):
        signalFrac = False
        signalCCI = 0
        entree = 0
        # Fractal croisemnt
        if ( pd_eurusd["CustFractSignal"].iloc[i-2] < pd_eurusd["CustFractSignalSMA"].iloc[i-2] and pd_eurusd["CustFractSignal"].iloc[i-1] > pd_eurusd["CustFractSignalSMA"].iloc[i-1] ) or ( pd_eurusd["CustFractSignal"].iloc[i-2] > pd_eurusd["CustFractSignalSMA"].iloc[i-2] and pd_eurusd["CustFractSignal"].iloc[i-1] < pd_eurusd["CustFractSignalSMA"].iloc[i-1] ):
            signalFrac = True
        #CCI signal
        if pd_eurusd["CCI_M5"].iloc[i-1] >= 100 and pd_eurusd["RSI_M5"].iloc[i-1] >= 70:
            signalCCI = 1
        elif pd_eurusd["CCI_M5"].iloc[i-1] <= -100 and pd_eurusd["RSI_M5"].iloc[i-1] <= 30:
            signalCCI = -1
        # Entrée
        # Baisse
        if signalCCI == 1 and signalFrac:

            capital -= mise
            if capital < 0:
                print("Pas assez de fonds: ", capital, " i: ", i)
                break
            R1, S1 = pivot(pd_eurusd["high_x"].iloc[i-1], pd_eurusd["low_x"].iloc[i-1], pd_eurusd["close_x"].iloc[i-1])
            entree = pd_eurusd["open_x"].iloc[i]
            for j in range(i+1, timer[_]):
                if pd_eurusd["high_x"].iloc[j] >= R1:
                    if pd_eurusd["spread_x"].iloc[j] <= spread:
                        gain = round( ((entree - (R1 + pd_eurusd["spread_x"].iloc[j] * 0.0001)) * 10_000) * ((0.0001 * 100_000 * lot) / entree), 2)
                        capital += round(mise + gain, 2)
                        gains = np.append(gains, gain)
                        trades += 1

                        end.append([i, j])
                        break
                elif pd_eurusd["low_x"].iloc[j] <= S1:
                    R1, S1 = pivot(pd_eurusd["high_x"].iloc[j-1], pd_eurusd["low_x"].iloc[j-1], pd_eurusd["close_x"].iloc[j-1])
            else:
                capital += mise
        # Hausse
        elif signalCCI == -1 and signalFrac:
            R1, S1 = pivot(pd_eurusd["high_x"].iloc[i-1], pd_eurusd["low_x"].iloc[i-1], pd_eurusd["close_x"].iloc[i-1])
            if pd_eurusd["spread_x"].iloc[i] <= spread:

                capital -= mise
                if capital < 0:
                    print("Pas assez de fonds: ", capital, " i: ", i)
                    break
                entree = pd_eurusd["open_x"].iloc[i] + pd_eurusd["spread_x"].iloc[i] * 0.0001
                for j in range(i+1, timer[_]):
                    if pd_eurusd["low_x"].iloc[j] <= S1: # or gain < -mise * 20 / 100:
                        gain = round( ((S1 - entree) * 10_000) * ((0.0001 * 100_000 * lot) / S1), 2)
                        capital += round(mise + gain, 2)
                        gains = np.append(gains, gain)
                        trades += 1

                        end.append([i, j])
                        break
                    elif pd_eurusd["high_x"].iloc[j] >= R1:
                        R1, S1 = pivot(pd_eurusd["high_x"].iloc[j-1], pd_eurusd["low_x"].iloc[j-1], pd_eurusd["close_x"].iloc[j-1])
                else:
                    capital += mise


print("Nb trades: ", trades)
print("Gains: ", gains.sum())
print("Capital: ", capital)
# print(end[-1])
# print(pd_eurusd["close_x"].iloc[end[-1][0]], " - ", end[-1][0], " *** ", pd_eurusd["close_x"].iloc[end[-1][1]], " - ", end[-1][1])

test = pd.DataFrame(gains)
test.describe()


Nb trades:  1
Gains:  -0.05
Capital:  99.95


,0
count,1.00
mean,-0.05
std,NaN
min,-0.05
25%,-0.05
50%,-0.05
75%,-0.05
max,-0.05


In [28]:
# A1 = 0
# A2 = 1

# SI(A1=0, "zero", "pouet")

# if A1 == 0:
#     print("zero")
# else:
#     print("pouet")
# Points pivots
# CCI_M5: trades 4296, gains 0.23414 => 0.000054509
# CCI_M30: trades 1809, gains 0.07854 => 0.000043416
# RSI_M5: trades 657, gains 0.04697 => 0.000071492

# Points pivots Fibonacci
# CCI_M5: trades 4296, gains 0.32318=>
# CCI_M30: trades 1809, gains 0.12582 =>
# RSI_M5: trades 657, gains 0.06588 =>

# 1 lot = 1 * 100_000 = (100_000 / 100_000) * 100_000
# 0.1 lot = 0.1 * 100_000 = (10_000 / 100_000) * 100_000

# 20 * 0.01 = 20 / 100 = 0.2
# 2 * 0.1 = 2 /10 = 0.2

# gains.sort()
print(gains)

test = pd.DataFrame(pd_eurusd, columns=["time", "close_x", "CCI_M5", "RSI_M5", "CustFractSignal", "CustFractSignalSMA"])
# print(test[test["time"] % 1800 == 1500][-50:])
print(test[-16:])


[-0.05]
           time  close_x      CCI_M5     RSI_M5  CustFractSignal  \
984  1616189700  108.885  -63.793957  50.847106         0.605202   
985  1616190000  108.890    7.332101  52.124223         0.544678   
986  1616190300  108.890   24.422442  52.124223         0.596259   
987  1616190600  108.882  -66.666667  49.839900         0.753599   
988  1616190900  108.890   71.428571  52.046519         0.742584   
989  1616191200  108.892  114.015152  52.593954         0.677903   
990  1616191500  108.896  200.000000  53.703834         0.699104   
991  1616191800  108.905  229.603316  56.130325         0.842299   
992  1616192100  108.901  193.573207  54.790206         1.023221   
993  1616192400  108.911  182.067077  57.456501         1.274362   
994  1616192700  108.904  118.369830  55.069386         1.361941   
995  1616193000  108.917  145.833333  58.436666         1.267109   
996  1616193300  108.916  128.681120  58.085067         1.076567   
997  1616193600  108.915  108.108108  57